# BSS-ANOVA eigendecomposition at increasing resolution (to later find asymptotic eigenvalue ratios)

Importing packages.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.optimize import curve_fit

Defining the BSS-ANOVA kernel main effect $\kappa_1$, outlined in equation 8 of [Fast variable selection makes Karhunen-Loève decomposed Gaussian process BSS-ANOVA a speedy and accurate choice for dynamic systems identification](docs/_static/arXiv.2205.13676v2.pdf),

$\kappa_1(x,x') = \mathcal{B}(x)\mathcal{B}_1(x') + \mathcal{B}_2(x)\mathcal{B}_2(x') + \frac{1}{24}\mathcal{B}_4(|x-x'|)$

where

$\begin{cases} \mathcal{B}_1(x) = x - \frac{1}{2} \\ \mathcal{B}_2(x) = x^2 - x + \frac{1}{6} \\ \mathcal{B}_4(x) = x^4 - 2x^3 + x^2 - \frac{1}{30} \end{cases}$

In [5]:
def b1(x):
    return x - 1/2

def b2(x):
    return x**2 - x + 1/6

def b4(x):
    return x**4 - 2*x**3 + x**2 - 1/30

def k1(xi, xj):
    return b1(xi)*b1(xj) + b2(xi)*b2(xj) - b4(np.abs(xi-xj))/24

Taking eigenvalues for increasing resolution of covariance matrix (i.e., BSS-ANOVA kernel). Because only 20 Bernoulli polynomials could be computed in MATLAB prior to significant rounding error in plots, only need first 20 eigenvalues.

In [22]:
n = 20  # number of Bernoulli polynomials (i.e., number of eigenvalues to save)
res_n = 5  # number of points to plot
res_lb = 1600  # lower bound (of plot)
res_ub = 2000  # upper bound (of plot)

eigvals = np.zeros([res_n, n])
res_x = np.linspace(res_lb, res_lb + np.round((res_ub-res_lb)/(res_n-1))*(res_n-1), res_n, dtype=int)
res_iter = 0
for res in res_x:
    x = np.linspace(0, 1, res)
    kernel = np.zeros([res, res])

    for i in range(res):
        for j in range(res):
            kernel[i, j] = k1(x[i], x[j])
    eigval, eigvec = np.linalg.eig(kernel)

    eigvals[res_iter, :] = eigval[:n]  # in future, plot columns which are basis function scales
    res_iter += 1

progress = np.concatenate([res_x[:, np.newaxis], eigvals], axis=1)
np.savetxt(f'current_progress_{res_lb}_{res_ub}.txt', progress)  # res points by basis function order (i.e., 'k' or eigenvalue id)

# !!! NOTE !!!
# Manually combine multiple 'current_progress_{res_lb}_{res_ub}.txt' files into single 'BSS-ANOVA_eigenvalues_for_20x20_thru_2000x2000.txt'

/tmp/ipykernel_28833/3318184751.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvals[res_iter, :] = eigval[:n]  # in future, plot columns which are basis function scales
